In [37]:
# Import libraries

import numpy as np
import pandas as pd
import itertools
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
# Training news data. 
data = pd.read_csv('../Resources/news.csv')

In [3]:
#  Note the last column denoting whether articles are fake or real
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
# Dropping the first column not needed.
data.drop("Unnamed: 0", axis=1, inplace=True)

In [5]:
# Lowercasing all in 'text' column.
data['text'] = data['text'].apply(lambda x: x.lower())

In [6]:
data.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"daniel greenfield, a shillman journalism fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,google pinterest digg linkedin reddit stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,u.s. secretary of state john f. kerry said mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— kaydee king (@kaydeeking) november 9, 2016 t...",FAKE
4,The Battle of New York: Why This Primary Matters,it's primary day in new york and front-runners...,REAL


In [7]:
# (rows, columns) returned
data.shape

(6335, 3)

In [13]:
labels = data['label']
labels.head()

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [14]:
# We expect the last column will be either FAKE or REAL.  Just confirming.
data['label'].unique()

array(['FAKE', 'REAL'], dtype=object)

In [15]:
# Doing futher sanity checking by counting number of each columns.
data = data.dropna()
data.count()

title    6335
text     6335
label    6335
dtype: int64

In [16]:
# Split the dataset
x_train, x_test, y_train, y_test = train_test_split(data['text'], labels, test_size=0.2, random_state=7)

In [18]:
# Initialize a TfidfVectorizer. 
# See for details https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

# Fit and transform train set, transform test set
tfidf_train = tfidf_vectorizer.fit_transform(x_train) 
tfidf_test = tfidf_vectorizer.transform(x_test)

In [22]:
# Initialize a PassiveAggressiveClassifier
pac = PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train, y_train)

# Predict on the test set and calculate accuracy
y_pred = pac.predict(tfidf_test)
score = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 92.42%


In [24]:
# Build confusion matrix
confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])

# Need to still add text for 591 True Postives, 588 ture Negitives, 
# 47 False Postives, and 41 False Negitives
# ToDo: Need to elaborate on how to interpret the results below.

array([[585,  53],
       [ 43, 586]])

In [25]:
y_test.head()

3534    REAL
6265    FAKE
3123    REAL
3940    REAL
2856    REAL
Name: label, dtype: object

In [31]:
# Set the Pandas column width not to truncate the long text lines.
pd.set_option('display.max_colwidth', None)
data.iloc[[3534, 6265], :]

,title,text,label
3534,"Hillary Clinton, Bernie Sanders: King's legacy is alive","a day after the candidates squared off in a fiery debate, they came to columbia, south carolina, and largely agreed that while king's impact can still be felt today, work still needs to be done to guarantee racial equality.\n\n""yes, the challenges we face are many, but so are the quiet heroes working in every corner of america today doing their part to make our country a better place,"" said the former secretary of state. ""i for one receive much inspiration from that simple fact.""\n\nthere was symbolism in the event organized by the naacp: in front of a statehouse that flew the confederate battle flag until it was taken down last year. all three candidates noted the flag being removed.\n\n""the flag is down but we are still here because that flag was just one piece of something bigger,"" clinton said. ""dr. king died with his work unfinished and it is up to us to see through.""\n\nsanders argued that king is not just a historic figure, but someone whose moral compass should guide people today. repeating the phrase ""i think if he were here today,"" sanders argued that if king were alive today, he would be supporting many of his presidential positions. ""as we celebrate his life it is terribly important to me that we don't just look at him as a museum figure, somebody in the past,"" the vermont senator said. ""it is important to me that we look at his vision, to see the america he wanted to see."" sanders and o'malley walked in the naacp sponsored march before the event, strolling down the streets of charleston as activists chanted. o'malley, whose birthday is monday, laughed when asked what he wanted for his birthday, telling reporters that he is hoping for ""beat expectations"" in iowa for his birthday.",REAL
6265,VIDEO : FBI SOURCES SAY INDICTMENT LIKELY FOR CLINTON – TruthFeed,"video : fbi sources say indictment likely for clinton video : fbi sources say indictment likely for clinton videos by truthfeednews november 3, 2016 \nbret baier: here’s the deal: we talked to two separate sources with intimate knowledge of the fbi investigations. one: the clinton foundation investigation is far more expansive than anybody has reported so far… several offices separately have been doing their own investigations. \ntwo: the immunity deal that cheryl mills and heather samuelson, two top aides to hillary clinton, got from the justice department in which it was beleived that the laptops they had, after a narrow review for classified materials, were going to be destroyed. we have been told that those have not been destroyed — they are at the fbi field office here on washington and are being exploited. . \nthree: the clinton foundation investigation is so expansive, they have interviewed and re-interviewed many people. they described the evidence they have as ‘a lot of it’ and said there is an ‘avalanche coming in every day.’ wikileaks and the new emails. \nthey are “actively and aggressively pursuing this case.” remember the foundation case is about accusations of pay-for-play… they are taking the new information and some of them are going back to interview people for the third time. as opposed to what has been written about the clinton foundation investigation, it is expansive. \nthe classified e-mail investigation is being run by the national security division of the fbi. they are currently combing through anthony weiner’s laptop. they are having some success — finding what they believe to be new emaisls, not duplicates, that have been transported through hillary clinton’s server. \nfinally, we learned there is a confidence from these sources that her server had been hacked. and that it was a 99% accuracy that it had been hacked by at least five foreign intelligence agencies, and that things had been taken from that… \nthere has been some angst about attorney general loretta lynch — what she has done or not done. she obviously did not impanel, or go to a

In [36]:
# fake = y_test['FAKE'].value_counts()
y_test.value_counts()

FAKE    638
REAL    629
Name: label, dtype: int64

In [43]:
# Saving then loading trained model using pickel lib. See https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/

pickle.dump(pac, open('saved_model', 'wb'))

loaded_model = pickle.load(open('saved_model', 'rb'))
tfidf_test = tfidf_vectorizer.transform(x_test)
# Predict on the test set and calculate accuracy
y_pred = loaded_model.predict(tfidf_test)
score = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(score*100,2)}%')


Accuracy: 92.42%
